In [40]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
path='/content/gdrive/My Drive/neural-insight/'

In [0]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D

from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import regularizers

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.initializers import glorot_normal
from keras.utils import np_utils
from math import ceil,floor
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

In [0]:
n_bits=15

In [0]:
X_train=np.load(path+'xtrain.npy').reshape(-1,n_bits)
y_train=np.load(path+'ytrain.npy').reshape(-1,2)
X_test=np.load(path+'xtest.npy').reshape(-1,n_bits)
y_test=np.load(path+'ytest.npy').reshape(-1,2)
X_train,y_train=shuffle(X_train,y_train)
X_test,y_test=shuffle(X_test,y_test)

In [0]:
X_train=np.load(path+'xtrain2.npy').reshape(-1,1)
y_train=np.load(path+'ytrain2.npy').reshape(-1,1)
X_test=np.load(path+'xtest2.npy').reshape(-1,1)
y_test=np.load(path+'ytest2.npy').reshape(-1,1)


[0,1] for prime
[1,0] for non prime

Precision of predicting a composite number is 1 i.e There are no false positives

In [116]:
model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(n_bits,)))
model.add(Dropout(0.4))


model.add(Dense(128,activation='relu'))

model.add(Dropout(0.4))

model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))
adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = ['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 256)               4096      
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_45 (Dense)             (None, 2)                 130       
Total params: 45,378
Trainable params: 45,378
Non-trainable params: 0
__________________________________________________

In [117]:
early_stop=keras.callbacks.callbacks.EarlyStopping(monitor='accuracy', min_delta=0, patience=40, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
reduce_lr=keras.callbacks.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
model.fit(X_train,y_train,epochs=200, batch_size=64,callbacks=[reduce_lr,early_stop])

Epoch 1/200
4000/4000 [==============================] - 0s 117us/step - loss: 0.5035 - accuracy: 0.7728
Epoch 2/200
4000/4000 [==============================] - 0s 77us/step - loss: 0.3727 - accuracy: 0.8508
Epoch 3/200
4000/4000 [==============================] - 0s 76us/step - loss: 0.3450 - accuracy: 0.8645
Epoch 4/200
4000/4000 [==============================] - 0s 75us/step - loss: 0.3311 - accuracy: 0.8662
Epoch 5/200
4000/4000 [==============================] - 0s 76us/step - loss: 0.3261 - accuracy: 0.8708
Epoch 6/200
4000/4000 [==============================] - 0s 75us/step - loss: 0.3182 - accuracy: 0.8737
Epoch 7/200
4000/4000 [==============================] - 0s 75us/step - loss: 0.3121 - accuracy: 0.8737
Epoch 8/200
4000/4000 [==============================] - 0s 76us/step - loss: 0.3114 - accuracy: 0.8748
Epoch 9/200
4000/4000 [==============================] - 0s 76us/step - loss: 0.3092 - accuracy: 0.8773
Epoch 10/200
4000/4000 [==============================] - 0s 78

In [121]:
model.evaluate(X_train,y_train)[1]

4000/4000 [==============================] - 0s 67us/step


0.8817499876022339

In [122]:
pred = model.predict_classes(X_test, verbose=0)
print(classification_report([np.where(r==1)[0][0] for r in y_test], pred))

              precision    recall  f1-score   support

           0       0.43      0.76      0.55       500
           1       0.02      0.00      0.01       500

    accuracy                           0.38      1000
   macro avg       0.22      0.38      0.28      1000
weighted avg       0.22      0.38      0.28      1000



In [107]:
print(X_test,pred)

[[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 1 ... 1 1 0]
 ...
 [1 0 1 ... 0 1 1]
 [0 1 1 ... 0 0 1]
 [1 1 1 ... 1 0 0]] [1 0 1 ... 1 1 1]


In [0]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        y += j<<i
    return y
nums=[bool2int(x[::-1]) for x in X_test]

In [120]:
for i in range(1000):
  print(nums[i],pred[i],y_test[i])

17337 0 [1 0]
21191 0 [0 1]
23814 0 [1 0]
20873 0 [0 1]
20129 0 [0 1]
20177 1 [0 1]
18367 0 [0 1]
20233 1 [0 1]
14270 0 [1 0]
17982 0 [1 0]
20815 0 [1 0]
30195 0 [1 0]
13825 0 [1 0]
21757 0 [0 1]
1004 1 [1 0]
3528 0 [1 0]
32340 0 [1 0]
20117 0 [0 1]
23796 1 [1 0]
20593 0 [0 1]
3780 0 [1 0]
5709 0 [1 0]
21557 0 [0 1]
18041 0 [0 1]
19717 1 [0 1]
30372 0 [1 0]
22193 0 [0 1]
735 0 [1 0]
29911 0 [1 0]
20407 0 [0 1]
18253 0 [0 1]
20717 0 [0 1]
3794 0 [1 0]
21813 0 [1 0]
19894 0 [1 0]
22164 0 [1 0]
21491 0 [0 1]
12314 0 [1 0]
9648 0 [1 0]
20641 0 [0 1]
2679 0 [1 0]
18919 1 [0 1]
7298 0 [1 0]
20599 0 [0 1]
12661 0 [1 0]
24425 0 [1 0]
21701 0 [0 1]
32556 0 [1 0]
22129 0 [0 1]
22123 0 [0 1]
19571 1 [0 1]
19763 0 [0 1]
10816 0 [1 0]
19031 0 [0 1]
19973 0 [0 1]
27473 0 [1 0]
18660 0 [1 0]
20161 1 [0 1]
18476 0 [1 0]
16667 0 [1 0]
18869 0 [0 1]
15893 0 [1 0]
20816 0 [1 0]
17881 0 [0 1]
19231 0 [0 1]
6261 0 [1 0]
22003 0 [0 1]
18701 0 [0 1]
18301 0 [0 1]
20983 1 [0 1]
16777 0 [1 0]
15784 0 [1 0]
209

In [0]:
from sklearn.neural_network import MLPRegressor,MLPClassifier

In [0]:
model = MLPRegressor(hidden_layer_sizes=(128,128,128,128,128,128,128,128,128,128),n_iter_no_change=200,random_state=581,learning_rate='adaptive',verbose=True,max_iter=200)

In [82]:
model.fit(X_train,y_train.ravel())

Iteration 1, loss = 18459245784.24534225
Iteration 2, loss = 916011887.72037065
Iteration 3, loss = 940682927.91360784
Iteration 4, loss = 1007231605.15223050
Iteration 5, loss = 1003319957.78973591
Iteration 6, loss = 1001481350.74837565
Iteration 7, loss = 997114138.88577974
Iteration 8, loss = 995729567.36030579
Iteration 9, loss = 961379958.51730621
Iteration 10, loss = 973054358.57994246
Iteration 11, loss = 971437989.32615185
Iteration 12, loss = 935947036.70375359
Iteration 13, loss = 8107750323.40449715
Iteration 14, loss = 838426049.64324963
Iteration 15, loss = 848994929.07969081
Iteration 16, loss = 877813252.47678280
Iteration 17, loss = 882051950.93676364
Iteration 18, loss = 892339717.94924581
Iteration 19, loss = 892902867.68447661
Iteration 20, loss = 898025894.04637706
Iteration 21, loss = 866374350.39391530
Iteration 22, loss = 869754768.44160950
Iteration 23, loss = 824261825.12934172
Iteration 24, loss = 820113188.32413065
Iteration 25, loss = 785451017.87329674
Ite

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(128, 128, 128, 128, 128, 128, 128, 128, 128,
                                 128),
             learning_rate='adaptive', learning_rate_init=0.001, max_fun=15000,
             max_iter=200, momentum=0.9, n_iter_no_change=200,
             nesterovs_momentum=True, power_t=0.5, random_state=581,
             shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
             verbose=True, warm_start=False)

In [83]:
model.score(X_test,y_test)

0.9994007416728249

In [67]:
X_train.shape

(200000, 1)

In [77]:
y_train

array([[      2],
       [      3],
       [      5],
       ...,
       [2750123],
       [2750131],
       [2750159]])

In [71]:
X_train

array([[     1],
       [     2],
       [     3],
       ...,
       [199998],
       [199999],
       [200000]])

In [85]:
model.predict(X_train)

array([4.19652337e+00, 5.31014987e+00, 6.06385092e+00, ...,
       2.74976992e+06, 2.74978456e+06, 2.74979921e+06])